# Matrix Addition Kernel
--------

In [1]:
!nvidia-smi

Mon Apr 28 12:00:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   35C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## CUDA

In [6]:
%%writefile matrix_addition.cu

#include <iostream>
#include <ctime>
#include <chrono>
#include <cstdlib>
#include <cuda_runtime.h>

__global__ void matrixAdd(const float* A, const float* B, float* C, int rows, int columns)
{
    // Get thread row index
    int row_index = blockIdx.y * blockDim.y + threadIdx.y;
    // Get thread column index
    int column_index = blockIdx.x * blockDim.x + threadIdx.x;
    // Check out of bounds
    if (row_index < rows && column_index < columns)
    {
        int index = row_index * columns + column_index;
        C[index] = A[index] + B[index];
    }
}

void matrixAddCPU(const float* A, const float* B, float* C, int rows, int columns)
{
    for (int row_index = 0; row_index < rows; row_index++)
    {
        for (int column_index = 0; column_index < columns; column_index++)
        {
            int index = row_index * columns + column_index;
            C[index] = A[index] + B[index];
        }
    }
}

void initialiseVectors(float *A, float *B, int N)
{
    srand(static_cast<unsigned int>(time(0)));

    for (int i = 0; i < N; i++)
    {
        A[i] = static_cast<float>(rand());
        B[i] = static_cast<float>(rand());
    }
}

// Experiment with auto instead of template later
template <typename Func>
double measureExecutionTime(Func func)
{
    auto start = std::chrono::high_resolution_clock::now();
    func();
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> duration = end - start;
    return duration.count();
}

bool compareResults(const float *A, const float *B, int N)
{
    for (int i = 0; i < N; i++)
    {
        if (fabs(A[i] - B[i]) > 1e-4)
        {
            std::cout << "Mismatch at index " << i << ": CPU=" << A[i] << " GPU=" << B[i] << std::endl;
            return false;
        }
    }
    return true;
}


int main()
{
    int num_rows = 1 << 13;
    int block_size_rows = 19;
    int num_columns = 1 << 13;
    int block_size_columns = 19;

    size_t size = num_rows * num_columns * sizeof(float);

    // Allocate memory on host
    float* A_host = (float*)malloc(size);
    float* B_host = (float*)malloc(size);
    float* C_mat_cpu = (float*)malloc(size);
    float* C_mat_gpu = (float*)malloc(size);

    // Initialise matrix
    initialiseVectors(A_host, B_host, num_rows * num_columns);

    // Measure CPU execution time
    double cpu_time = measureExecutionTime([&]()
    {
        matrixAddCPU(A_host, B_host, C_mat_cpu, num_rows, num_columns);
    });
    std::cout << "CPU execution time: " << cpu_time << " ms" << '\n';

    // Allocate memory on device
    float* A_device;
    float* B_device;
    float* C_device;

    cudaMalloc((void**)&A_device, size);
    cudaMalloc((void**)&B_device, size);
    cudaMalloc((void**)&C_device, size);

    // Copy data from host to device
    cudaMemcpy(A_device, A_host, size, cudaMemcpyHostToDevice);
    cudaMemcpy(B_device, B_host, size, cudaMemcpyHostToDevice);

    // Define grid
    int num_blocks_rows = (num_rows + block_size_rows - 1) / block_size_rows;
    int num_blocks_columns = (num_columns + block_size_columns - 1) / block_size_columns;

    dim3 block(block_size_columns, block_size_rows, 1);
    dim3 grid(num_blocks_columns, num_blocks_rows, 1);

    // Measure GPU execution time
    double gpu_time = measureExecutionTime([&]()
    {
        matrixAdd<<<grid, block>>>(A_device, B_device, C_device, num_rows, num_columns);
        cudaDeviceSynchronize();
    });
    std::cout << "GPU execution time: " << gpu_time << " ms" << '\n';

    // Copy results from device to host
    cudaMemcpy(C_mat_gpu, C_device, size, cudaMemcpyDeviceToHost);

    bool success = compareResults(C_mat_cpu, C_mat_gpu, num_rows * num_columns);
    std::cout << (success ? "CPU and GPU results match!" : "Results mismatch!") << '\n';

    // Free device memory
    cudaFree(A_device);
    cudaFree(B_device);
    cudaFree(C_device);

    // Free host memory
    free(A_host);
    free(B_host);
    free(C_mat_cpu);
    free(C_mat_gpu);

    return 0;
}

Overwriting matrix_addition.cu


In [7]:
!nvcc -arch=sm_89 matrix_addition.cu -o matrix_addition
!./matrix_addition

CPU execution time: 349.847 ms
GPU execution time: 3.42599 ms
CPU and GPU results match!


## Triton

In [13]:
import torch
import triton
import triton.language as tl
import time

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# Initialise matrices
size = 1 << 13
x = torch.rand((size, size), device=DEVICE)
y = torch.rand((size, size), device=DEVICE)
x_cpu = torch.rand((size, size), device="cpu")
y_cpu = torch.rand((size, size), device="cpu")

@triton.jit
def matrix_add_kernel(
    x_ptr, y_ptr, output_ptr,
    M, N, # num_rows, num_columns
    stride_xm, stride_xn,
    stride_ym, stride_yn,
    stride_om, stride_on,
    BLOCK_SIZE_M: tl.constexpr, # block_size_rows
    BLOCK_SIZE_N: tl.constexpr # block_size_columns
    ):

  pid_m = tl.program_id(0) # x
  pid_n = tl.program_id(1) # y

  row_start = pid_m * BLOCK_SIZE_M
  column_start = pid_n * BLOCK_SIZE_N

  rows = row_start + tl.arange(0, BLOCK_SIZE_M)
  columns = column_start + tl.arange(0, BLOCK_SIZE_N)

  # Compute flat memory offsets for each (row, col) pair in the tile
  # using broadcasting to generate the full 2D grid of indices
  offsets_x = rows.expand_dims(1) * stride_xm + columns.expand_dims(0) * stride_xn
  offsets_y = rows.expand_dims(1) * stride_ym + columns.expand_dims(0) * stride_yn
  offsets_o = rows.expand_dims(1) * stride_om + columns.expand_dims(0) * stride_on

  # Allow access to elements where both row and column indices are in bounds
  mask = (rows.expand_dims(1) < M) & (columns.expand_dims(0) < N)

  # Load elements of the tiles from DRAM, masking out out-of-bound elements with 0.0
  x_tile = tl.load(x_ptr + offsets_x, mask=mask, other=0.0)
  y_tile = tl.load(y_ptr + offsets_y, mask=mask, other=0.0)

  output_tile = x_tile + y_tile

  # Write result back to DRAM
  tl.store(output_ptr + offsets_o, output_tile, mask=mask)


def matrix_add(x, y):
  output = torch.empty_like(x)

  assert x.device == DEVICE and y.device == DEVICE and output.device == DEVICE, "Tensors must be on CUDA"
  assert x.shape == y.shape and x.shape == output.shape, "Tensors must have identical dimension"

  M, N = output.shape

  stride_xm, stride_xn = x.stride()
  stride_ym, stride_yn = y.stride()
  stride_om, stride_on = output.stride()

  grid = lambda meta: (triton.cdiv(M, meta["BLOCK_SIZE_M"]),
                       triton.cdiv(N, meta["BLOCK_SIZE_N"]))

  matrix_add_kernel[grid](
      x, y, output,
      M, N,
      stride_xm, stride_xn,
      stride_ym, stride_yn,
      stride_om, stride_on,
      BLOCK_SIZE_M=32,
      BLOCK_SIZE_N=32
      )

  return output

# Warm up and cache kernel
_ = matrix_add(x, y)

# Measure Triton execution time
torch.cuda.synchronize()
start = time.perf_counter()
output_triton = matrix_add(x, y)
torch.cuda.synchronize()
end = time.perf_counter()
triton_time = (end - start) * 1000

# Measure PyTorch GPU execution time
torch.cuda.synchronize()
start = time.perf_counter()
output_torch_gpu = x + y
torch.cuda.synchronize()
end = time.perf_counter()
pytorch_gpu_time = (end - start) * 1000

# Measure Pytorch CPU execution time
start = time.perf_counter()
output_torch_cpu = x_cpu + y_cpu
end = time.perf_counter()
pytorch_cpu_time = (end - start) * 1000

# Check correctness
max_diff = torch.max(torch.abs(output_triton - output_torch_gpu))
assert torch.allclose(output_triton, output_torch_gpu, atol=1e-5), "Mismatch with PyTorch!"

print(f"Triton time:       {triton_time:.3f} ms")
print(f"PyTorch GPU time:  {pytorch_gpu_time:.3f} ms")
print(f"PyTorch CPU time:  {pytorch_cpu_time:.3f} ms")
print(f'The maximum difference between torch and triton is '
      f'{torch.max(torch.abs(output_torch_gpu - output_triton))}')


cuda:0
Triton time:       3.964 ms
PyTorch GPU time:  3.548 ms
PyTorch CPU time:  42.756 ms
The maximum difference between torch and triton is 0.0
